In [57]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [ ]:
#First get a diverse set of songs from Spotify:

In [ ]:
#Global X, hits from cultures around the world, 100 songs

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [ ]:
#get title, artist, and uri for each song in playlist

In [20]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [22]:
gx_playlist = get_playlist_tracks('37i9dQZF1DWUxHPh2rEiHr')

In [53]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

def get_details_from_track(track):
    return flatten((track["name"],artist["name"], track["uri"]) for artist in track["artists"])

def get_details_from_playlist_item(playlist_item):
  return get_details_from_track(playlist_item['track'])

def get_details_from_playlist(input_playlist):
  return list(map(get_details_from_playlist_item,input_playlist))

In [55]:
gx_details = get_details_from_playlist(gx_playlist)

In [114]:
def make_df_from_playlist_details(playlist_details):
    titles = [song[0] for song in playlist_details]
    artists = [song[1] for song in playlist_details]
    uris = [(song[2].split(":"))[-1] for song in playlist_details]
    df = pd.DataFrame({'title':titles, 'artist':artists, 'uri':uris})
    
    return df

In [115]:
gx = make_df_from_playlist_details(gx_details)

In [116]:
gx

,title,artist,uri
0,Fever,Dua Lipa,3wJVBz7gvGTwTWVAxFa6ny
1,Rotate,Becky G,6wwT3PgG3eFuKCpxzjMJFs
2,Aquí (feat. Ozuna & Soolking),AriBeatz,0259eSt61D349yZvwily88
3,G-Shit (feat. Blockboi Twitch),Sidhu Moose Wala,5uuVGnkZEFHBz8unViC3iA
4,M.I.A (feat. Jackson Wang),Afgan,0Uuw4Topqtjut1xQFMXEzU
...,...,...,...
95,Where You Wanna Be,R3HAB,7qbjS6uKZlv7K08CNcEqYR
96,O Fată Obişnuită,AMI,44zAieIynvghU0rDvfhQnm
97,Chama Ela (feat. Pedro Sampaio),Lexa,14kRWkCWBYfQqILflEcpSJ
98,Killem,Angelo King,3m8FZtwaOCVMQvzh8nrEyp


In [ ]:
#get audio features and add them to the df

In [166]:
audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'time_signature']

def add_audio_features(df):
    df['features'] = None
    for i in range(len(df)):
        song_features = sp.audio_features(df.uri[i])
        df['features'][i] = song_features[0]
        sleep(randint(1,3))
    for f in audio_features:
        df[f] = [df['features'][i][f] for i in range(len(df))]
    df.drop(['features'], axis=1, inplace=True)
    return df

In [162]:
gx_with_audio = add_audio_features(gx)

In [165]:
gx_with_audio.drop(['features'], axis=1, inplace=True)
gx_with_audio

,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Fever,Dua Lipa,3wJVBz7gvGTwTWVAxFa6ny,0.868,0.693,6,-7.052,0,0.1810,0.0597,0.003990,0.1060,0.396,114.999,4
1,Rotate,Becky G,6wwT3PgG3eFuKCpxzjMJFs,0.910,0.856,11,-2.082,0,0.1570,0.0346,0.000223,0.0624,0.771,109.029,4
2,Aquí (feat. Ozuna & Soolking),AriBeatz,0259eSt61D349yZvwily88,0.535,0.755,6,-5.354,0,0.0559,0.0420,0.000000,0.1520,0.311,191.931,4
3,G-Shit (feat. Blockboi Twitch),Sidhu Moose Wala,5uuVGnkZEFHBz8unViC3iA,0.658,0.761,0,-6.645,1,0.2570,0.0531,0.000000,0.2160,0.843,165.543,4
4,M.I.A (feat. Jackson Wang),Afgan,0Uuw4Topqtjut1xQFMXEzU,0.723,0.458,6,-5.572,0,0.1700,0.2560,0.000000,0.0879,0.357,113.954,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Where You Wanna Be,R3HAB,7qbjS6uKZlv7K08CNcEqYR,0.766,0.698,3,-7.549,0,0.2280,0.3290,0.000000,0.0835,0.204,123.926,4
96,O Fată Obişnuită,AMI,44zAieIynvghU0rDvfhQnm,0.491,0.628,6,-6.699,1,0.0630,0.3180,0.000000,0.0987,0.456,146.953,4
97,Chama Ela (feat. Pedro Sampaio),Lexa,14kRWkCWBYfQqILflEcpSJ,0.888,0.841,6,-1.933,1,0.1470,0.4600,0.000091,0.0568,0.964,149.994,4
98,Killem,Angelo King,3m8FZtwaOCVMQvzh8nrEyp,0.751,0.880,5,-3.079,0,0.1090,0.0599,0.000000,0.3270,0.928,100.047,4


In [167]:
#make function to automate whole process:
def make_audio_features_df_from_playlistID(playlist_id):
    
    def get_playlist_tracks(playlist_id):
        results = sp.user_playlist_tracks("spotify",playlist_id)
        tracks = results['items']
        while results['next']!=None:
            results = sp.next(results)
            tracks = tracks + results['items']
            sleep(randint(1,3))
        return tracks
    
    def flatten(input_list):
        return [item for sublist in input_list for item in sublist]

    def get_details_from_track(track):
        return flatten((track["name"],artist["name"], track["uri"]) for artist in track["artists"])

    def get_details_from_playlist_item(playlist_item):
        return get_details_from_track(playlist_item['track'])

    def get_details_from_playlist(input_playlist):
        return list(map(get_details_from_playlist_item,input_playlist))
    
    def make_df_from_playlist_details(playlist_details):
        titles = [song[0] for song in playlist_details]
        artists = [song[1] for song in playlist_details]
        uris = [song[2] for song in playlist_details]
        return pd.DataFrame({'title':titles, 'artist':artists, 'uri':uris})
    
    playlist = get_playlist_tracks(playlist_id)
    details = get_details_from_playlist(playlist)
    df = make_df_from_playlist_details(details)
    
    audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'time_signature']

    def add_audio_features(df):
        df['features'] = None
        for i in range(len(df)):
            song_features = sp.audio_features(df.uri[i])
            df['features'][i] = song_features[0]
            sleep(randint(1,3))
        for f in audio_features:
            df[f] = [df['features'][i][f] for i in range(len(df))]
        df.drop(['features'], axis=1, inplace=True)
        return df
    
    df_with_audio_features = add_audio_features(df)
    
    return df_with_audio_features

In [177]:
#make function to automate whole process:
def make_audio_features_df_from_playlistID(playlist_id):
    
    playlist = get_playlist_tracks(playlist_id)
    details = get_details_from_playlist(playlist)
    df = make_df_from_playlist_details(details)
    
    audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
            'liveness', 'valence', 'tempo', 'time_signature']
    
    df_with_audio_features = add_audio_features(df)
    
    return df_with_audio_features

In [168]:
def add_playlist(playlist_id, songs_df):
    df = make_audio_features_df_from_playlistID(playlist_id)
    new_df = pd.concat([songs_df, df], axis=0)
    new_df.drop_duplicates(subset=['title'], keep='first', inplace=True)
    new_df.reset_index(inplace=True, drop=True)
    return new_df

In [ ]:
#90's road trip 37i9dQZF1DX76cnmxfAAhD
#70's rock anthems 37i9dQZF1DWWwzidNQX6jx
#spotify and chill 37i9dQZF1DX7ZnTv0GKubq
#spotify japan 37i9dQZF1DX9vYRBO9gjDe
#millenial prom 37i9dQZF1DXc4yak2w3qgm
#rapcaviar 37i9dQZF1DX0XUsuxWHRQd
#peaceful piano 37i9dQZF1DX4sWSpwq3LiO
#rb 37i9dQZF1DX4SBhb3fqCJd
#rock classics 37i9dQZF1DWXRqgorJj26U
#00's 37i9dQZF1DX4o1oenSJRJd
#90's 37i9dQZF1DXbTxeAdrVG2l
#80's 37i9dQZF1DX4UtSsGT1Sbe
#70's 37i9dQZF1DWTJ7xPn4vNaz

#60's 37i9dQZF1DXaKIA8E7WcJj
#50's 37i9dQZF1DWSV3Tk4GO2fq
#gold school 37i9dQZF1DWVA1Gq4XHa6U
#get turnt 37i9dQZF1DWY4xHQp97fN6
#african heat 37i9dQZF1DWYkaDif7Ztbp
#alternative hip-hop 37i9dQZF1DWTggY0yqBxES
#new alt 37i9dQZF1DX82GYcclJ3Ug
#you + me 37i9dQZF1DX6mvEU1S6INL


In [169]:
songs = add_playlist('37i9dQZF1DX76cnmxfAAhD', gx_with_audio)

In [172]:
songs = add_playlist('37i9dQZF1DWWwzidNQX6jx', songs)

In [174]:
songs = add_playlist('37i9dQZF1DX7ZnTv0GKubq', songs)

In [176]:
songs = add_playlist('37i9dQZF1DX9vYRBO9gjDe', songs)

In [179]:
songs = add_playlist('37i9dQZF1DXc4yak2w3qgm', songs)

In [181]:
songs = add_playlist('37i9dQZF1DX0XUsuxWHRQd', songs)

In [183]:
songs = add_playlist('37i9dQZF1DX4sWSpwq3LiO', songs)

In [185]:
songs = add_playlist('37i9dQZF1DX4SBhb3fqCJd', songs)

In [187]:
songs = add_playlist('37i9dQZF1DWXRqgorJj26U', songs)

In [189]:
songs = add_playlist('37i9dQZF1DX4o1oenSJRJd', songs)

In [191]:
songs = add_playlist('37i9dQZF1DXbTxeAdrVG2l', songs)

In [193]:
songs = add_playlist('37i9dQZF1DX4UtSsGT1Sbe', songs)

In [196]:
songs = add_playlist('37i9dQZF1DWTJ7xPn4vNaz', songs)

In [195]:
songs.to_csv('songs.csv')

In [ ]:
#I will add more songs later, but I ran out of time tonight...